In [81]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
# from sklearn.grid_search import 

In [82]:
df = pd.read_csv('train.csv')
print(df.shape)
df.head()

(26729, 10)


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [83]:
df.columns

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 10 columns):
AnimalID          26729 non-null object
Name              19038 non-null object
DateTime          26729 non-null object
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
dtypes: object(10)
memory usage: 2.0+ MB


In [85]:
import re
def age2Days(x):
    m = re.search(r'(?P<num>\d+) (?P<gran>(day|days|week|weeks|month|months|year|years))', x)
    if m != None:
        n = int(m.group('num'))
        gran = m.group('gran')
        if gran == 'week' or gran == 'weeks':
            n *= 7
        if gran == 'month' or gran == 'months':
            n *= 30 # assume 30 day months
        if gran == 'year' or gran == 'years':
            n *= 12*30
        return n
    else:
        return None


In [86]:
X_train = df[['AnimalType', 'SexuponOutcome', 'AgeuponOutcome']]
y_train = df['OutcomeType']

In [87]:
print(X_train.shape, y_train.shape)
# idx = X_train.loc[X_train.AgeuponOutcome.isnull() == False, ]
# X_train = X_train.loc[idx.index.to_list(),]
# y_train = y_train[idx.index.to_list()]
# print(X_train.shape, y_train.shape)
# print(X_train.isnull().values.any())
# print(y_train.isnull().values.any())
X_train = X_train.dropna()
idx = X_train.loc[X_train.AgeuponOutcome.isnull() == False, ]
X_train = X_train.loc[idx.index.to_list(),]
y_train = y_train[idx.index.to_list()]
print(X_train.shape, y_train.shape)
X_train.info()

(26729, 3) (26729,)
(26710, 3) (26710,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26710 entries, 0 to 26728
Data columns (total 3 columns):
AnimalType        26710 non-null object
SexuponOutcome    26710 non-null object
AgeuponOutcome    26710 non-null object
dtypes: object(3)
memory usage: 834.7+ KB


In [88]:
from sklearn.preprocessing import LabelEncoder

at_enc = LabelEncoder()
so_enc = LabelEncoder()

X_train.AnimalType = at_enc.fit_transform(X_train.AnimalType)
X_train.SexuponOutcome = so_enc.fit_transform(X_train.SexuponOutcome)

In [89]:
# convert the age given in weeks, months and years to days
X_train.AgeuponOutcome = X_train.AgeuponOutcome.map(age2Days)

In [90]:
print(X_train.info())
X_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26710 entries, 0 to 26728
Data columns (total 3 columns):
AnimalType        26710 non-null int32
SexuponOutcome    26710 non-null int32
AgeuponOutcome    26710 non-null int64
dtypes: int32(2), int64(1)
memory usage: 626.0 KB
None


,AnimalType,SexuponOutcome,AgeuponOutcome
0,1,2,360
1,0,3,360
2,1,2,720
3,0,1,21
4,1,2,720


In [91]:
#DecisionTreeClassifier is does not handle missing values. So the NaNs should be repalced with some legal values
# X_train = X_train.astype({'Name' : 'category', 'OutcomeSubtype' : 'category', 'AnimalType' : 'category'
#                , 'SexuponOutcome' : 'category', 'AgeuponOutcome' : 'category', 'Breed' : 'category'
#                , 'Color' : 'category'})

# X_train.Name.cat.add_categories('Noname', inplace=True)
# X_train.OutcomeSubtype.cat.add_categories('Nosubtype', inplace=True)
# X_train.SexuponOutcome.cat.add_categories('Nosex', inplace=True)
# X_train.AgeuponOutcome.cat.add_categories('0 age', inplace=True)

# X_train.Name[X_train.Name.isnull() == True] = 'Noname'
# X_train.OutcomeSubtype[X_train.OutcomeSubtype.isnull() == True] = 'Nosubtype'
# X_train.SexuponOutcome[X_train.SexuponOutcome.isnull() == True] = 'Nosex'
# X_train.AgeuponOutcome[X_train.AgeuponOutcome.isnull() == True] = '0 age'

# X_train = X_train.astype({'AnimalType' : 'category'
#                , 'SexuponOutcome' : 'category', 'AgeuponOutcome' : 'category'})

X_trn, X_tst, y_trn, y_tst = train_test_split(X_train, y_train, random_state=25)

In [92]:
from sklearn.impute import SimpleImputer
print(X_train.shape)
print(y_train.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline = Pipeline([
#                 ('imp', sklearn.impute.SimpleImputer(strategy='constant', fill_value='Dummy')),
#                 ('enc', sklearn.preprocessing.OrdinalEncoder()),
#                 ('enc', sklearn.preprocessing.LabelEncoder()),    
                ('clf', DecisionTreeClassifier(criterion='entropy'))
           ])
parameters = {
    'clf__max_depth': (100, 150, 155, 160),
    'clf__min_samples_split': (2, 3),
    'clf__min_samples_leaf': (1, 2, 3)
}

(26710, 3)
(26710,)


In [93]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='accuracy')
# grid_search.fit(X_train, y_train)
grid_search.fit(X_trn, y_trn)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s


Best score: 0.625
Best parameters set:
	clf__max_depth: 100
	clf__min_samples_leaf: 2
	clf__min_samples_split: 2


[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    3.6s finished


In [94]:
# test
# dt = pd.read_csv('test.csv')

# X_test = dt[['AnimalType', 'SexuponOutcome', 'AgeuponOutcome']]

# X_test.AnimalType = at_enc.transform(X_test.AnimalType)
# X_test.SexuponOutcome = so_enc.transform(X_test.SexuponOutcome)
# X_test.AgeuponOutcome = ao_enc.transform(X_test.AgeuponOutcome)

from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn import tree
import pydotplus

_,z = grid_search.best_estimator_.steps[0]
dot_data = StringIO()
tree.export_graphviz(z
                    , out_file = dot_data
                    , feature_names = ['AnimalType', 'SexuponOutcome', 'AgeuponOutcome']
                    , filled=True, rounded=True, impurity=False)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [111]:
# pred_tst = grid_search.predict(X_tst)
pred_tst = z.predict(X_tst)
print(classification_report(y_tst, pred_tst))

                 precision    recall  f1-score   support

       Adoption       0.66      0.82      0.73      2753
           Died       0.00      0.00      0.00        50
     Euthanasia       0.47      0.07      0.12       372
Return_to_owner       0.46      0.44      0.45      1218
       Transfer       0.72      0.65      0.68      2285

      micro avg       0.64      0.64      0.64      6678
      macro avg       0.46      0.40      0.40      6678
   weighted avg       0.63      0.64      0.62      6678



In [117]:
from sklearn.metrics import accuracy_score, log_loss
y_prob = z.predict_proba(X_tst)
print("log_loss:")
print(log_loss(y_tst, y_prob))

log_loss:
1.0777451725395446


In [108]:
from sklearn.metrics import confusion_matrix
# pred_trn = grid_search.predict(X_trn)
pred_trn = z.predict(X_trn)
cm_trn = confusion_matrix(y_trn, pred_trn)
print("Confusion matrix-Training")
print(cm_trn)

Confusion matrix-Training
[[6507    0    8  973  528]
 [  19    6    2    9  111]
 [ 213    1  108  312  547]
 [1468    0   18 1623  458]
 [1806    5   47  712 4551]]


In [99]:
print(classification_report(y_trn, pred_trn))

                 precision    recall  f1-score   support

       Adoption       0.65      0.81      0.72      8016
           Died       0.50      0.04      0.08       147
     Euthanasia       0.59      0.09      0.16      1181
Return_to_owner       0.45      0.46      0.45      3567
       Transfer       0.73      0.64      0.68      7121

      micro avg       0.64      0.64      0.64     20032
      macro avg       0.58      0.41      0.42     20032
   weighted avg       0.64      0.64      0.62     20032



In [118]:
from sklearn.metrics import accuracy_score, log_loss
y_prob = z.predict_proba(X_trn)
print("log_loss:")
print(log_loss(y_trn, y_prob))

log_loss:
0.8526635073055194


In [104]:
acc_trn = np.trace(cm_trn)/sum(sum(cm_trn))
print("Accuracy-Training:")
print(acc_trn)

Accuracy-Training:
0.63872803514377


In [107]:
cm_tst = confusion_matrix(y_tst, pred_tst)
print("Confusion matrix-Test:")
print(cm_tst)

Confusion matrix-Test:
[[2249    0    3  331  170]
 [   3    0    1    3   43]
 [  67    1   25   96  183]
 [ 500    0    8  540  170]
 [ 567    4   16  216 1482]]


In [105]:
acc_tst = np.trace(cm_tst)/sum(sum(cm_tst))
print("Accuracy-Test:")
print(acc_tst)

Accuracy-Test:
0.6433063791554358
